In [143]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [144]:
HEADERS = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Accept-Language': 'en-US,en;q=0.5',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:130.0) Gecko/20100101 Firefox/130.0'
}

In [145]:
def get_content(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        return BeautifulSoup(response.content, 'lxml')
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return None

In [146]:
data_list=[]

def get_url(query,page=1):
    while True:
        url=f'https://fitnessprogramer.com/exercise-primary-muscle/{query}/page/{page}/'
        # print(f"Scraping page {page} at URL: {url}")

        gifs = get_content(url).find_all('div', class_='thumbnails')

        for gif in gifs:
            title = gif.find('img')['alt']
            src = gif.find('img')['src']

            data_list.append({'title': title, 'src': src})
            

        next_page = get_content(url).find('a', class_='next') 
        if next_page and 'href' in next_page.attrs:
            page += 1
        else:
            # print("No more pages to scrape.")
            break       
    

In [147]:
data_list.clear() # clear the list before appending new data

In [148]:
def scrape(query):
    
    data=get_url(query)
    data=pd.DataFrame(data_list)

    print(data.head(10).to_string(index=False)) # for testing purposes
    # data.to_csv(f'{query}_exercises.csv', index=False) # uncomment this line to save the data to a csv file

In [149]:
# neck exercises
scrape('neck')

                        title                                                                                             src
Weighted Lateral Neck Flexion https://fitnessprogramer.com/wp-content/uploads/2022/02/Lying-Weighted-Lateral-Neck-Flexion.gif
Weighted Lying Neck Extension       https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Extension.gif
  Weighted Lying Neck Flexion         https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Flexion.gif
              Gittleson Shrug     https://fitnessprogramer.com/wp-content/uploads/2022/01/Dumbbell-Seated-Gittleson-Shrug.gif
        Diagonal Neck Stretch       https://fitnessprogramer.com/wp-content/uploads/2021/07/Diagonal-Neck-Stretch-360x360.png
        Neck Rotation Stretch               https://fitnessprogramer.com/wp-content/uploads/2021/07/Rotating-Neck-Stretch.gif
         Neck Flexion Stretch        https://fitnessprogramer.com/wp-content/uploads/2021/07/Neck-Flexion-Stretch-360x

In [150]:
# trapezius exercises
scrape('neck')

                        title                                                                                             src
Weighted Lateral Neck Flexion https://fitnessprogramer.com/wp-content/uploads/2022/02/Lying-Weighted-Lateral-Neck-Flexion.gif
Weighted Lying Neck Extension       https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Extension.gif
  Weighted Lying Neck Flexion         https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Flexion.gif
              Gittleson Shrug     https://fitnessprogramer.com/wp-content/uploads/2022/01/Dumbbell-Seated-Gittleson-Shrug.gif
        Diagonal Neck Stretch       https://fitnessprogramer.com/wp-content/uploads/2021/07/Diagonal-Neck-Stretch-360x360.png
        Neck Rotation Stretch               https://fitnessprogramer.com/wp-content/uploads/2021/07/Rotating-Neck-Stretch.gif
         Neck Flexion Stretch        https://fitnessprogramer.com/wp-content/uploads/2021/07/Neck-Flexion-Stretch-360x

In [151]:
# shoulders exercises
scrape('shoulders')

                        title                                                                                             src
Weighted Lateral Neck Flexion https://fitnessprogramer.com/wp-content/uploads/2022/02/Lying-Weighted-Lateral-Neck-Flexion.gif
Weighted Lying Neck Extension       https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Extension.gif
  Weighted Lying Neck Flexion         https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Flexion.gif
              Gittleson Shrug     https://fitnessprogramer.com/wp-content/uploads/2022/01/Dumbbell-Seated-Gittleson-Shrug.gif
        Diagonal Neck Stretch       https://fitnessprogramer.com/wp-content/uploads/2021/07/Diagonal-Neck-Stretch-360x360.png
        Neck Rotation Stretch               https://fitnessprogramer.com/wp-content/uploads/2021/07/Rotating-Neck-Stretch.gif
         Neck Flexion Stretch        https://fitnessprogramer.com/wp-content/uploads/2021/07/Neck-Flexion-Stretch-360x

In [152]:
# chest exercises
scrape('chest')

                        title                                                                                             src
Weighted Lateral Neck Flexion https://fitnessprogramer.com/wp-content/uploads/2022/02/Lying-Weighted-Lateral-Neck-Flexion.gif
Weighted Lying Neck Extension       https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Extension.gif
  Weighted Lying Neck Flexion         https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Flexion.gif
              Gittleson Shrug     https://fitnessprogramer.com/wp-content/uploads/2022/01/Dumbbell-Seated-Gittleson-Shrug.gif
        Diagonal Neck Stretch       https://fitnessprogramer.com/wp-content/uploads/2021/07/Diagonal-Neck-Stretch-360x360.png
        Neck Rotation Stretch               https://fitnessprogramer.com/wp-content/uploads/2021/07/Rotating-Neck-Stretch.gif
         Neck Flexion Stretch        https://fitnessprogramer.com/wp-content/uploads/2021/07/Neck-Flexion-Stretch-360x

In [153]:
# back exercises
scrape('back')

                        title                                                                                             src
Weighted Lateral Neck Flexion https://fitnessprogramer.com/wp-content/uploads/2022/02/Lying-Weighted-Lateral-Neck-Flexion.gif
Weighted Lying Neck Extension       https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Extension.gif
  Weighted Lying Neck Flexion         https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Flexion.gif
              Gittleson Shrug     https://fitnessprogramer.com/wp-content/uploads/2022/01/Dumbbell-Seated-Gittleson-Shrug.gif
        Diagonal Neck Stretch       https://fitnessprogramer.com/wp-content/uploads/2021/07/Diagonal-Neck-Stretch-360x360.png
        Neck Rotation Stretch               https://fitnessprogramer.com/wp-content/uploads/2021/07/Rotating-Neck-Stretch.gif
         Neck Flexion Stretch        https://fitnessprogramer.com/wp-content/uploads/2021/07/Neck-Flexion-Stretch-360x

In [154]:
# erector-spinae exercises
scrape('erector-spinae')

                        title                                                                                             src
Weighted Lateral Neck Flexion https://fitnessprogramer.com/wp-content/uploads/2022/02/Lying-Weighted-Lateral-Neck-Flexion.gif
Weighted Lying Neck Extension       https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Extension.gif
  Weighted Lying Neck Flexion         https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Flexion.gif
              Gittleson Shrug     https://fitnessprogramer.com/wp-content/uploads/2022/01/Dumbbell-Seated-Gittleson-Shrug.gif
        Diagonal Neck Stretch       https://fitnessprogramer.com/wp-content/uploads/2021/07/Diagonal-Neck-Stretch-360x360.png
        Neck Rotation Stretch               https://fitnessprogramer.com/wp-content/uploads/2021/07/Rotating-Neck-Stretch.gif
         Neck Flexion Stretch        https://fitnessprogramer.com/wp-content/uploads/2021/07/Neck-Flexion-Stretch-360x

In [155]:
# biceps exercises
scrape('biceps')

                        title                                                                                             src
Weighted Lateral Neck Flexion https://fitnessprogramer.com/wp-content/uploads/2022/02/Lying-Weighted-Lateral-Neck-Flexion.gif
Weighted Lying Neck Extension       https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Extension.gif
  Weighted Lying Neck Flexion         https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Flexion.gif
              Gittleson Shrug     https://fitnessprogramer.com/wp-content/uploads/2022/01/Dumbbell-Seated-Gittleson-Shrug.gif
        Diagonal Neck Stretch       https://fitnessprogramer.com/wp-content/uploads/2021/07/Diagonal-Neck-Stretch-360x360.png
        Neck Rotation Stretch               https://fitnessprogramer.com/wp-content/uploads/2021/07/Rotating-Neck-Stretch.gif
         Neck Flexion Stretch        https://fitnessprogramer.com/wp-content/uploads/2021/07/Neck-Flexion-Stretch-360x

In [156]:
# triceps exercises
scrape('triceps')

                        title                                                                                             src
Weighted Lateral Neck Flexion https://fitnessprogramer.com/wp-content/uploads/2022/02/Lying-Weighted-Lateral-Neck-Flexion.gif
Weighted Lying Neck Extension       https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Extension.gif
  Weighted Lying Neck Flexion         https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Flexion.gif
              Gittleson Shrug     https://fitnessprogramer.com/wp-content/uploads/2022/01/Dumbbell-Seated-Gittleson-Shrug.gif
        Diagonal Neck Stretch       https://fitnessprogramer.com/wp-content/uploads/2021/07/Diagonal-Neck-Stretch-360x360.png
        Neck Rotation Stretch               https://fitnessprogramer.com/wp-content/uploads/2021/07/Rotating-Neck-Stretch.gif
         Neck Flexion Stretch        https://fitnessprogramer.com/wp-content/uploads/2021/07/Neck-Flexion-Stretch-360x

In [157]:
# forearm exercises
scrape('forearm')

                        title                                                                                             src
Weighted Lateral Neck Flexion https://fitnessprogramer.com/wp-content/uploads/2022/02/Lying-Weighted-Lateral-Neck-Flexion.gif
Weighted Lying Neck Extension       https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Extension.gif
  Weighted Lying Neck Flexion         https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Flexion.gif
              Gittleson Shrug     https://fitnessprogramer.com/wp-content/uploads/2022/01/Dumbbell-Seated-Gittleson-Shrug.gif
        Diagonal Neck Stretch       https://fitnessprogramer.com/wp-content/uploads/2021/07/Diagonal-Neck-Stretch-360x360.png
        Neck Rotation Stretch               https://fitnessprogramer.com/wp-content/uploads/2021/07/Rotating-Neck-Stretch.gif
         Neck Flexion Stretch        https://fitnessprogramer.com/wp-content/uploads/2021/07/Neck-Flexion-Stretch-360x

In [158]:
# abs exercises
scrape('abs')

                        title                                                                                             src
Weighted Lateral Neck Flexion https://fitnessprogramer.com/wp-content/uploads/2022/02/Lying-Weighted-Lateral-Neck-Flexion.gif
Weighted Lying Neck Extension       https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Extension.gif
  Weighted Lying Neck Flexion         https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Flexion.gif
              Gittleson Shrug     https://fitnessprogramer.com/wp-content/uploads/2022/01/Dumbbell-Seated-Gittleson-Shrug.gif
        Diagonal Neck Stretch       https://fitnessprogramer.com/wp-content/uploads/2021/07/Diagonal-Neck-Stretch-360x360.png
        Neck Rotation Stretch               https://fitnessprogramer.com/wp-content/uploads/2021/07/Rotating-Neck-Stretch.gif
         Neck Flexion Stretch        https://fitnessprogramer.com/wp-content/uploads/2021/07/Neck-Flexion-Stretch-360x

In [159]:
# leg exercises
scrape('leg')

                        title                                                                                             src
Weighted Lateral Neck Flexion https://fitnessprogramer.com/wp-content/uploads/2022/02/Lying-Weighted-Lateral-Neck-Flexion.gif
Weighted Lying Neck Extension       https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Extension.gif
  Weighted Lying Neck Flexion         https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Flexion.gif
              Gittleson Shrug     https://fitnessprogramer.com/wp-content/uploads/2022/01/Dumbbell-Seated-Gittleson-Shrug.gif
        Diagonal Neck Stretch       https://fitnessprogramer.com/wp-content/uploads/2021/07/Diagonal-Neck-Stretch-360x360.png
        Neck Rotation Stretch               https://fitnessprogramer.com/wp-content/uploads/2021/07/Rotating-Neck-Stretch.gif
         Neck Flexion Stretch        https://fitnessprogramer.com/wp-content/uploads/2021/07/Neck-Flexion-Stretch-360x

In [160]:
# calf exercises
scrape('calf')

                        title                                                                                             src
Weighted Lateral Neck Flexion https://fitnessprogramer.com/wp-content/uploads/2022/02/Lying-Weighted-Lateral-Neck-Flexion.gif
Weighted Lying Neck Extension       https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Extension.gif
  Weighted Lying Neck Flexion         https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Flexion.gif
              Gittleson Shrug     https://fitnessprogramer.com/wp-content/uploads/2022/01/Dumbbell-Seated-Gittleson-Shrug.gif
        Diagonal Neck Stretch       https://fitnessprogramer.com/wp-content/uploads/2021/07/Diagonal-Neck-Stretch-360x360.png
        Neck Rotation Stretch               https://fitnessprogramer.com/wp-content/uploads/2021/07/Rotating-Neck-Stretch.gif
         Neck Flexion Stretch        https://fitnessprogramer.com/wp-content/uploads/2021/07/Neck-Flexion-Stretch-360x

In [161]:
# hips exercises
scrape('hips')

                        title                                                                                             src
Weighted Lateral Neck Flexion https://fitnessprogramer.com/wp-content/uploads/2022/02/Lying-Weighted-Lateral-Neck-Flexion.gif
Weighted Lying Neck Extension       https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Extension.gif
  Weighted Lying Neck Flexion         https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Flexion.gif
              Gittleson Shrug     https://fitnessprogramer.com/wp-content/uploads/2022/01/Dumbbell-Seated-Gittleson-Shrug.gif
        Diagonal Neck Stretch       https://fitnessprogramer.com/wp-content/uploads/2021/07/Diagonal-Neck-Stretch-360x360.png
        Neck Rotation Stretch               https://fitnessprogramer.com/wp-content/uploads/2021/07/Rotating-Neck-Stretch.gif
         Neck Flexion Stretch        https://fitnessprogramer.com/wp-content/uploads/2021/07/Neck-Flexion-Stretch-360x

In [162]:
# cardio exercises
scrape('cardio')

                        title                                                                                             src
Weighted Lateral Neck Flexion https://fitnessprogramer.com/wp-content/uploads/2022/02/Lying-Weighted-Lateral-Neck-Flexion.gif
Weighted Lying Neck Extension       https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Extension.gif
  Weighted Lying Neck Flexion         https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Flexion.gif
              Gittleson Shrug     https://fitnessprogramer.com/wp-content/uploads/2022/01/Dumbbell-Seated-Gittleson-Shrug.gif
        Diagonal Neck Stretch       https://fitnessprogramer.com/wp-content/uploads/2021/07/Diagonal-Neck-Stretch-360x360.png
        Neck Rotation Stretch               https://fitnessprogramer.com/wp-content/uploads/2021/07/Rotating-Neck-Stretch.gif
         Neck Flexion Stretch        https://fitnessprogramer.com/wp-content/uploads/2021/07/Neck-Flexion-Stretch-360x

In [163]:
# full-body exercises
scrape('full-body')

                        title                                                                                             src
Weighted Lateral Neck Flexion https://fitnessprogramer.com/wp-content/uploads/2022/02/Lying-Weighted-Lateral-Neck-Flexion.gif
Weighted Lying Neck Extension       https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Extension.gif
  Weighted Lying Neck Flexion         https://fitnessprogramer.com/wp-content/uploads/2021/08/Weighted-Lying-Neck-Flexion.gif
              Gittleson Shrug     https://fitnessprogramer.com/wp-content/uploads/2022/01/Dumbbell-Seated-Gittleson-Shrug.gif
        Diagonal Neck Stretch       https://fitnessprogramer.com/wp-content/uploads/2021/07/Diagonal-Neck-Stretch-360x360.png
        Neck Rotation Stretch               https://fitnessprogramer.com/wp-content/uploads/2021/07/Rotating-Neck-Stretch.gif
         Neck Flexion Stretch        https://fitnessprogramer.com/wp-content/uploads/2021/07/Neck-Flexion-Stretch-360x